<a href="https://colab.research.google.com/github/ddecosmo-dev/thread-checker/blob/main/threadCheckerApplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
#@title  fastener Classifier GUI
# ---------------------------------------------------------------------------------
# Section 1: Setup and Imports
# ---------------------------------------------------------------------------------
# Core libraries for machine learning and image processing
import tensorflow as tf
import numpy as np
from PIL import Image
import io
import json

# Libraries for building the interactive GUI in Colab
import ipywidgets as widgets
from ipywidgets import Layout
from google.colab import drive
from IPython.display import display
import matplotlib.pyplot as plt

print("Libraries imported successfully!")

##constants
IMG_HEIGHT = 224
IMG_WIDTH = 224

# ---------------------------------------------------------------------------------
# Section 2: Connect to Google Drive
# ---------------------------------------------------------------------------------
# This will prompt you for authorization to access your Google Drive files.
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

#drive.mount("/content/drive", force_remount=True)

Libraries imported successfully!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!


In [66]:
#import data from training
# ---------------------------------------------------------------------------------
# Section 3: Configuration & Model Loading (from JSON)
# ---------------------------------------------------------------------------------
# --- IMPORTANT: UPDATE THESE TWO PATHS ---

# 1. Path to the JSON file containing your class structure map.
CLASS_MAP_PATH = '/content/drive/MyDrive/24-679 AI ML Projects/Project 0/class_map.json'

# 2. Path to your trained model file.
MODEL_PATH = '/content/drive/MyDrive/24-679 AI ML Projects/Project 0/thread_checker_model.keras'
# --- End of Configuration ---


In [67]:
# Load the class map from the JSON file
try:
    with open(CLASS_MAP_PATH, 'r') as f:
        class_map = json.load(f)
    # Create the flat CLASS_NAMES list for the model's output index.
    # It sorts the map by index (0, 1, 2...) and gets the corresponding key.
    CLASS_NAMES = [key for key, value in sorted(class_map.items(), key=lambda item: item[1])]
    print(f"Successfully loaded {len(CLASS_NAMES)} class names from JSON map.")
except FileNotFoundError:
    print(f"ERROR: Class map file not found at {CLASS_MAP_PATH}. Please create it first.")
    CLASS_NAMES = []
except Exception as e:
    print(f"An error occurred while loading the JSON map: {e}")
    CLASS_NAMES = []

Successfully loaded 3 class names from JSON map.


In [68]:
# Load the CNN model
model = None
if CLASS_NAMES: # Only proceed if the class map loaded successfully
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        print(f"✅ Model loaded successfully from {MODEL_PATH}")
    except Exception as e:
        print(f"🚨 ERROR: Could not load the model file at {MODEL_PATH}. {e}")

✅ Model loaded successfully from /content/drive/MyDrive/24-679 AI ML Projects/Project 0/thread_checker_model.keras


In [69]:
# --- V E R I F I C A T I O N   S T E P ---
if model and CLASS_NAMES:
    model_output_units = model.output_shape[-1]
    num_loaded_classes = len(CLASS_NAMES)
    if model_output_units == num_loaded_classes:
        print(f"✅ Verification successful: Model and class map are in sync ({num_loaded_classes} classes).")
    else:
        print("❌ CRITICAL ERROR: Model and class map are out of sync!")
        model = None

✅ Verification successful: Model and class map are in sync (3 classes).


In [70]:
# ---------------------------------------------------------------------------------
# Section 4: Image Preprocessing Function
# ---------------------------------------------------------------------------------
def preprocess_image(image_bytes):
    img = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = tf.keras.utils.img_to_array(img)
    img_tensor = np.expand_dims(img_array, axis=0) # Add batch dimension
    return img_tensor


In [71]:
# ---------------------------------------------------------------------------------
# Section 5: Build the GUI Components
# ---------------------------------------------------------------------------------
header = widgets.HTML("<h1>Fastener Classification Engine</h1>"
                      "<p>Upload an image to identify the fastener type and specifications.</p>")

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Photo',
    style={'description_width': 'initial'},
    layout=Layout(width='250px')
)

# Use an Output widget to hold the plot
output_plot = widgets.Output()

In [72]:
# ---------------------------------------------------------------------------------
# Section 6: Prediction Logic (Event Handler)
# ---------------------------------------------------------------------------------
def on_upload_change(change):
    if not change.new or model is None:
        return

    # Clear previous plot
    output_plot.clear_output()

    uploaded_file = change.new[0]
    file_bytes = uploaded_file['content']

    # Preprocess the image (without normalization, as it's in the model)
    processed_tensor = preprocess_image(file_bytes)

    # Run prediction
    predictions = model.predict(processed_tensor)
    score = np.max(predictions[0])
    pred_index = np.argmax(predictions[0])
    pred_class_string = CLASS_NAMES[pred_index]

    # Parse the result string for a cleaner title
    parsed_title = pred_class_string.replace('_', ' ').replace('/', ' - ').title()
    final_title = f'Prediction: {parsed_title}\nConfidence: {score:.2%}'

    # Display the results in the output widget
    with output_plot:
        # Display the uploaded image as a plot
        img_to_plot = Image.open(io.BytesIO(file_bytes))
        plt.figure(figsize=(5, 5))
        plt.imshow(img_to_plot)
        plt.title(final_title)
        plt.axis('off')
        plt.show()

In [73]:
# ---------------------------------------------------------------------------------
# Section 7: Link Logic to GUI and Display
# ---------------------------------------------------------------------------------
uploader.observe(on_upload_change, names='value')

app_layout = widgets.VBox([
    header,
    uploader,
    output_plot
])

print("\nGUI is ready. Please upload an image.")
display(app_layout)


GUI is ready. Please upload an image.


KeyError: 0